# Hands on examples from 
[[MlMastery Arcticle on the LSTM TS Subject]](https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/#:~:text=Multivariate%20time%20series%20data%20means%20data%20where%20there%20is%20more)

In [9]:
import numpy as np

In [10]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [11]:
in_seq1.shape

(9,)

In [12]:
in_seq1

array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [13]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

In [14]:
in_seq1.shape

(9, 1)

In [15]:
in_seq1

array([[10],
       [20],
       [30],
       [40],
       [50],
       [60],
       [70],
       [80],
       [90]])

In [16]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [21]:
# split Fn
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [22]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [25]:
X[0], y[0]

(array([[10, 15],
        [20, 25],
        [30, 35]]),
 65)

In [27]:
# first three elements
for i in range(3):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105


# Using LSTM on the data

In [32]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten

In [34]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________


In [36]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [43]:
# demonstrate prediction
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input_r = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input_r, verbose=0)
print(yhat)

[[206.34424]]


In [44]:
x_input

array([[ 80,  85],
       [ 90,  95],
       [100, 105]])

In [45]:
x_input_r

array([[[ 80,  85],
        [ 90,  95],
        [100, 105]]])